In [1]:
#this is to try out converting the pdf files to ndjson
#importing libraries needed
import pandas as pd
import PyPDF2
import json
from pathlib import Path
import sys
import time




In [2]:
import logging

#creating loggers
logger = logging.getLogger()
logger.setLevel(logging.INFO)

#creating handlers
handlers = []

#creating console handler
ch = logging.StreamHandler()
ch.setLevel(logging.WARNING)

#creating file handler
fh = logging.FileHandler('test.log')
fh.setLevel(logging.INFO)

#adding handlers
handlers.append(ch)
handlers.append(fh)

# adding handlers to logger
for handler in handlers:
    logger.addHandler(handler)

In [3]:
#Setting the file path
proj_dir_parent = Path.cwd().parent
proj_dir = str(proj_dir_parent)

sys.path.append(proj_dir)


In [16]:
!wikiextractor -o "$proj_dir"/data/raw/output --json "$proj_dir"/data/raw/arwiki-latest-pages-articles-multistream.xml.bz2



INFO: Preprocessing '/Users/moutazelias/Desktop/Desktop/Github/UAE-Law-RAG/data/raw/arwiki-latest-pages-articles-multistream.xml.bz2' to collect template definitions: this may take some time.
INFO: Preprocessed 100000 pages
INFO: Preprocessed 200000 pages
INFO: Preprocessed 300000 pages
INFO: Preprocessed 400000 pages
INFO: Preprocessed 500000 pages
INFO: Preprocessed 600000 pages
INFO: Preprocessed 700000 pages
INFO: Preprocessed 800000 pages
INFO: Preprocessed 900000 pages
INFO: Preprocessed 1000000 pages
INFO: Preprocessed 1100000 pages
INFO: Preprocessed 1200000 pages
INFO: Preprocessed 1300000 pages
INFO: Preprocessed 1400000 pages
INFO: Preprocessed 1500000 pages
INFO: Preprocessed 1600000 pages
INFO: Preprocessed 1700000 pages
INFO: Preprocessed 1800000 pages
INFO: Preprocessed 1900000 pages
INFO: Preprocessed 2000000 pages
INFO: Preprocessed 2100000 pages
INFO: Preprocessed 2200000 pages
INFO: Preprocessed 2300000 pages
INFO: Preprocessed 2400000 pages
INFO: Preprocessed 250000

In [4]:
#Function to conver PDF in input dir to jsonl in output
def pdf_to_json(input_path, output_path):

    #open input path and get dir
    input_pa = Path(input_path)

    # for file in input_path:
    for f in input_pa.iterdir():

        if f.suffix == ".pdf":
            pdf_path = f

            pdf = PyPDF2.PdfReader(pdf_path)
            for i in range(len(pdf.pages)):
                page = pdf.pages[i]
                text = page.extract_text()

                temp = {"title":f"{f.name}" , "page":f"{i}","text":f"{text}"}

                with open(output_path, 'a') as o:
                    json.dump(temp, o)
                    o.write('\n')
    

In [5]:
pdf_to_json(input_path=f"{proj_dir}/PDF docs/", output_path=f"{proj_dir}/data/raw/output/PDF/pdfjsonl.jsonl")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/moutazelias/Desktop/Desktop/Github/UAE-Law-RAG/data/raw/output/PDF/pdfjsonl.jsonl'

In [6]:
from src.preprocessing.Preprocessing import folder_to_json

In [7]:
folder = proj_dir_parent / 'data/raw/output'
folder_out = proj_dir_parent / 'data/consolidated/'
folder_to_json(folder, folder_out, 'ar_wiki')

Processing:   0%|          | 0/6745 [00:00<?, ?file/s]

Wiki processed in 59.15 seconds!


In [4]:
from tqdm.auto import tqdm
import pickle
from haystack.nodes import PreProcessor


In [9]:
proj_dir = Path.cwd().parent
print(proj_dir)

/Users/moutazelias/Desktop/Desktop/Github/UAE-Law-RAG


In [10]:
files_in = list((proj_dir / 'data/consolidated').glob('*.ndjson'))
folder_out = proj_dir / 'data/processed'
folder_out_str = str(folder_out)


In [11]:
with open(files_in[0], 'r') as f:
    articles = [json.loads(line) for line in f]

In [12]:
PP = PreProcessor(
    clean_whitespace = True,
    clean_header_footer = False,
    clean_empty_lines = True,
    remove_substrings = None,
    split_by='word',
    split_length = 200,
    split_overlap = 50,
    split_respect_sentence_boundary = True,
    tokenizer_model_folder = None,
    id_hash_keys = None,
    progress_bar = True,
    add_page_number = False,
    max_chars_check = 10000
)

In [13]:
%time
documents = PP.process(articles)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.01 µs


Preprocessing:  79%|███████▉  | 78782/100000 [00:51<00:24, 855.24docs/s] Document eca1c79de6ca898903e5db0a44a54803 is 24659 characters long after preprocessing, where the maximum length should be 10000. Something might be wrong with the splitting, check the document affected to prevent issues at query time. This document will be now hard-split at 10000 chars recursively.
Document 8a59ff0086cc7772046f54a065566ff9 is 14659 characters long after preprocessing, where the maximum length should be 10000. Something might be wrong with the splitting, check the document affected to prevent issues at query time. This document will be now hard-split at 10000 chars recursively.
Preprocessing: 100%|██████████| 100000/100000 [01:05<00:00, 1534.77docs/s]


In [14]:
os.cpu_count()  

8

In [16]:
%%time
import os
import concurrent.futures as cf

def parallel_preprocessing(articles):
    # Utility function to divide the articles into smaller chunks
    def chunkify(lst, n):
        """Yield successive n-sized chunks from lst."""
        for i in range(0, len(lst), n):
            yield lst[i:i + n]

    # Size of each chunk. Adjust based on your needs.
    CHUNK_SIZE = 10000  
    article_chunks = list(chunkify(articles, CHUNK_SIZE))

    # Number of processes to run in parallel.
    # Use all available CPUs, but you can reduce the number if you wish to leave some CPUs free.
    NUM_PROCESSES = os.cpu_count()  

    with cf.ProcessPoolExecutor(max_workers=NUM_PROCESSES) as executor:
        documents_list = list(executor.map(PP.process, article_chunks))

    # Flatten the documents_list to get a single list of documents
    documents = [doc for sublist in documents_list for doc in sublist]
    return documents

documents = parallel_preprocessing(articles)

TypeError: ProcessPoolExecutor.__init__() got an unexpected keyword argument 'num_workers'